# TensorFlow变量的创建

TensorFlow的变量创建方式有两种，分别是tf.Variable和tf.get_variable，官方推荐使用tf.get_variable

## 两种方法定义

In [1]:
import tensorflow as tf

In [2]:
var = tf.Variable(tf.truncated_normal([3]))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(var))

[-1.8268476  0.2766269 -0.4346284]


In [3]:
var = tf.get_variable(name="var", shape=[3], initializer=tf.truncated_normal_initializer())
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(var))

[-0.18380766  0.7491091  -0.4326347 ]


In [4]:
var.name

'var:0'

In [5]:
weights = tf.Variable(tf.truncated_normal([784, 200]), name="weights")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(weights))

[[-8.5189956e-01  5.7471699e-01 -1.4136952e+00 ...  3.9799005e-01
  -3.1257191e-01 -2.4561206e-01]
 [ 9.4724971e-01  3.6756513e-01 -1.9106454e-01 ... -1.0788462e+00
  -6.7944193e-01  1.7225336e-01]
 [-1.1388873e+00  9.3900853e-01  3.4499478e-01 ...  1.4734310e+00
   3.4052056e-01 -3.8110232e-01]
 ...
 [-5.2085507e-01  1.0625203e+00  3.4610018e-01 ...  3.6608580e-01
   5.4061002e-01  5.5843723e-01]
 [-1.6051920e-01 -7.0722454e-05  1.9430110e+00 ...  1.8024760e+00
  -9.8369545e-01  3.5922137e-01]
 [-6.7796922e-01 -3.9240080e-01  1.4075945e-01 ... -7.0440727e-01
   7.7036953e-01  1.1048136e+00]]


In [6]:
weights.name

'weights:0'

In [7]:
weights = tf.get_variable(name="weights", shape=[784, 200], initializer=tf.truncated_normal_initializer())
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(weights))

[[ 0.76234174  0.36796165 -0.32125506 ... -0.27060705 -1.364285
   0.45150134]
 [ 0.4740744  -0.03633748 -0.07992953 ... -0.56561065  1.6337496
   0.7799617 ]
 [ 0.18745598 -1.4875408   1.1839578  ... -0.16558436  1.1458405
  -0.597676  ]
 ...
 [-0.7874381   0.32065526 -0.77521235 ... -1.159896    0.04889259
   1.0680661 ]
 [ 1.9263655   1.107359    0.46303895 ... -1.6908878  -0.43912092
  -1.8427316 ]
 [-1.1783707   1.3051972   1.4505495  ... -0.46290568  0.44194236
  -0.0049569 ]]


In [8]:
weights.name

'weights_1:0'

## 注意事项
没有就创建，有就报错

In [9]:
a = tf.get_variable(name="v", shape=[1])

In [10]:
b = tf.get_variable(name="v", shape=[1])

ValueError: Variable v already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-9-270739f72022>", line 1, in <module>
    a = tf.get_variable(name="v", shape=[1])
  File "/home/caolei/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/caolei/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 3185, in run_ast_nodes
    if (yield from self.run_code(code, result)):


## 共享变量

In [11]:
with tf.variable_scope("one"):
    a = tf.get_variable(name="v", shape=[2])
with tf.variable_scope("one"):
    b = tf.get_variable(name="v", shape=[2])

ValueError: Variable one/v already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-11-711a94c1dcfc>", line 2, in <module>
    a = tf.get_variable(name="v", shape=[2])
  File "/home/caolei/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/caolei/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 3185, in run_ast_nodes
    if (yield from self.run_code(code, result)):


In [12]:
with tf.variable_scope("one", reuse=True):
    c = tf.get_variable(name="v", shape=[2])

In [13]:
assert a == c

In [18]:
train_ds = tf.data.experimental.CsvDataset("./train.csv", [tf.string, tf.int32])

In [17]:
def _parse_image(file, label):
    img_string = tf.read_file(file)
    img = tf.image.decode_png(img_string)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.reshape(img, [28, 28, 1])
    label = tf.cast(label, tf.int32)
    return img, label

In [19]:
train_ds = train_ds.map(_parse_image)
train_ds = train_ds.shuffle(48000).batch(100)

In [20]:
train_ds.output_shapes

(TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(1)]),
 TensorShape([Dimension(None)]))

In [22]:
handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(handle, train_ds.output_types, train_ds.output_shapes)
data_x, data_y = iterator.get_next()

In [23]:
data_x

<tf.Tensor 'IteratorGetNext:0' shape=(?, 28, 28, 1) dtype=float32>

In [24]:
data_y

<tf.Tensor 'IteratorGetNext:1' shape=(?,) dtype=int32>

In [25]:
def weight_variable(shape):
    return tf.get_variable(name="weights", shape=shape, initializer=tf.truncated_normal_initializer())

In [26]:
def bias_variable(shape):
    return tf.get_variable(name="biases", shape=shape, initializer=tf.constant_initializer(0.1))

In [29]:
def conv_layer(inputs, filter_size, input_dim, output_dim, layer_name, use_pooling=True, act=tf.nn.relu):
    with tf.variable_scope(layer_name):
        weights = weight_variable(shape=[filter_size, filter_size, input_dim, output_dim])
        biases = bias_variable(shape=[output_dim])
    layer = tf.nn.conv2d(input=inputs, filter=weights, strides=[1, 1, 1, 1], padding="SAME")
    layer = tf.nn.bias_add(layer, biases)
    if use_pooling:
        layer = tf.nn.max_pool(value=layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
    layer = act(layer)
    return layer, weights

In [30]:
layer_conv1, weights_conv1 = conv_layer(inputs=data_x, filter_size=5, input_dim=1, output_dim=20, layer_name="layer1")

In [31]:
layer_conv1

<tf.Tensor 'Relu:0' shape=(?, 14, 14, 20) dtype=float32>

In [1]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_files = [os.path.join("./train_tfrecords/", i) for i in os.listdir("./train_tfrecords/")]
val_files = [os.path.join("test_tfrecords/", i) for i in os.listdir("./test_tfrecords/")]

In [3]:
print(train_files[:3])
print(val_files[:3])

['./train_tfrecords/train.tfrecord-045', './train_tfrecords/train.tfrecord-002', './train_tfrecords/train.tfrecord-033']
['test_tfrecords/test.tfrecord-003', 'test_tfrecords/test.tfrecord-001', 'test_tfrecords/test.tfrecord-002']


In [4]:
def _parser_func(record):
    keys_to_features = {"img_raw": tf.FixedLenFeature((), tf.string, default_value=""), "label": tf.FixedLenFeature((), tf.int64, default_value=tf.zeros([], tf.int64))}
    parsed = tf.parse_single_example(record, keys_to_features)
    img_decoded = tf.decode_raw(parsed["img_raw"], tf.uint8)
    img_reshaped = tf.reshape(img_decoded, [28, 28, 1])
    img_converted = tf.image.convert_image_dtype(img_reshaped, tf.float32)
    label = tf.cast(parsed["label"], tf.int32)
    return img_converted, label

In [5]:
train_ds = tf.data.TFRecordDataset(train_files).map(_parser_func).shuffle(60000).batch(100)
val_ds = tf.data.TFRecordDataset(val_files).map(_parser_func).shuffle(10000).batch(100)

In [6]:
print(train_ds)
print(val_ds)

<BatchDataset shapes: ((?, 28, 28, 1), (?,)), types: (tf.float32, tf.int32)>
<BatchDataset shapes: ((?, 28, 28, 1), (?,)), types: (tf.float32, tf.int32)>


In [7]:
train_val_iterator = tf.data.Iterator.from_structure(train_ds.output_types, train_ds.output_shapes)
train_iterator = train_val_iterator.make_initializer(train_ds)
val_iterator = train_val_iterator.make_initializer(val_ds)

In [8]:
handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(handle, train_ds.output_types, train_ds.output_shapes)
data_x, data_y = iterator.get_next()

In [9]:
def weight_variable(layer_name, shape):
    with tf.variable_scope(layer_name, reuse=tf.AUTO_REUSE):
        weights = tf.get_variable(name="weights", shape=shape, initializer=tf.truncated_normal_initializer())
    return weights


def bias_variable(layer_name, shape):
    with tf.variable_scope(layer_name, reuse=tf.AUTO_REUSE):
        biases = tf.get_variable(name="biases", shape=shape, initializer=tf.constant_initializer(0.1))
    return biases

In [11]:
def conv_layer(inputs, filter_size, input_dim, output_dim, layer_name, padding="SAME", use_pooling=True):
    weights = weight_variable(layer_name, shape=[filter_size, filter_size, input_dim, output_dim])
    biases = bias_variable(layer_name, shape=[output_dim])
    layer = tf.nn.conv2d(input=inputs, filter=weights, strides=[1, 1, 1, 1], padding=padding)
    layer = tf.nn.bias_add(layer, biases)
    if use_pooling:
        layer = tf.nn.max_pool(value=layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
    layer = tf.nn.relu(layer)
    return layer, weights


def flatten_layer(inputs):
    inputs_shape = inputs.get_shape()
    num_features = inputs_shape[1:4].num_elements()
    layer = tf.reshape(inputs, [-1, num_features])
    return layer, num_features


def fc_layer(inputs, input_dim, output_dim, layer_name, use_relu=True):
    weights = weight_variable(layer_name, shape=[input_dim, output_dim])
    biases = bias_variable(layer_name, shape=[output_dim])
    layer = tf.add(tf.matmul(inputs, weights), biases)
    if use_relu:
        layer = tf.nn.relu(layer)
    return layer

In [16]:
class MyModel:
    def __init__(self, data_x, data_y):
        self.n_classes = 10
        self._create_architecture(data_x, data_y)
        
    def _create_model(self, x):
        self.layer_conv1, self.weights_conv1 = conv_layer(inputs=x, filter_size=5, input_dim=1, output_dim=20, padding="VALID", layer_name="layer_conv1")
        self.layer_conv2, self.weights_conv2 = conv_layer(inputs=self.layer_conv1, filter_size=5, input_dim=20, output_dim=50, layer_name="layer_conv2")
        self.layer_flat, self.num_features = flatten_layer(self.layer_conv2)
        self.layer_fc1 = fc_layer(inputs=self.layer_flat, input_dim=self.num_features, output_dim=500, layer_name="fc1")
        self.logits = fc_layer(inputs=self.layer_fc1, input_dim=500, output_dim=self.n_classes, use_relu=False, layer_name="logits")
        return self.logits
    
    def _create_architecture(self, x, y):
        self.logits = self._create_model(x)
        predictions = tf.argmax(self.logits, 1, output_type=tf.int32)
        self.loss = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(labels=y, logits=self.logits))
        self.train_step = tf.train.AdamOptimizer().minimize(self.loss)
        self.accuracy = tf.reduce_sum(tf.cast(tf.equal(predictions, y), tf.float32))

In [17]:
model = MyModel(data_x, data_y)

In [20]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
train_val_string = sess.run(train_val_iterator.string_handle())

In [21]:
from tqdm import tqdm

In [22]:
for epoch in range(5):
    train_acc, train_loss = 0, 0
    val_acc, val_loss = 0, 0
    
    sess.run(train_iterator)
    try:
        with tqdm(total=60000) as pbar:
            while True:
                _, loss, acc = sess.run([model.train_step, model.loss, model.accuracy], feed_dict={handle: train_val_string})
                train_loss += loss
                train_acc += acc
                pbar.update(100)
    except tf.errors.OutOfRangeError:
        pass
    
    sess.run(val_iterator)
    try:
        while True:
            loss, acc = sess.run([model.loss, model.accuracy], feed_dict={handle: train_val_string})
            val_loss += loss
            val_acc += acc
    except tf.errors.OutOfRangeError:
        pass
    
    print("\nEpoch: {}".format(epoch + 1))
    print("Train accuracy: {:.4f}, loss: {:.4f}".format(train_acc / 60000, train_loss / 60000))
    print("Val accuracy: {:.4f}, loss: {:.4f}".format(val_acc / 10000, val_loss / 10000))

  0%|          | 0/60000 [00:00<?, ?it/s]


Epoch: 1
Train accuracy: 0.8444, loss: 7.6966
Val accuracy: 0.9317, loss: 2.0755


  0%|          | 0/60000 [00:00<?, ?it/s]


Epoch: 2
Train accuracy: 0.9461, loss: 1.4476
Val accuracy: 0.9454, loss: 1.4044


  0%|          | 0/60000 [00:00<?, ?it/s]


Epoch: 3
Train accuracy: 0.9634, loss: 0.8253
Val accuracy: 0.9619, loss: 0.8930


  0%|          | 0/60000 [00:00<?, ?it/s]


Epoch: 4
Train accuracy: 0.9709, loss: 0.5775
Val accuracy: 0.9682, loss: 0.6766


100%|██████████| 60000/60000 [00:43<00:00, 1365.54it/s]



Epoch: 5
Train accuracy: 0.9775, loss: 0.3928
Val accuracy: 0.9710, loss: 0.5809
